# LASSO via coordinate descent

These notes and example follow from the course in Regression from University of Washington on Coursera. We implement our own LASSO solver via coordinate descent. We will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

In [109]:
import turicreate
import numpy as np

### Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [61]:
sales = turicreate.SFrame('home_data.sframe/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float and then to int, before using it below
sales['floors'] = sales['floors'].astype(float).astype(int)

### Useful functions

We convert the SFrame into a 2D Numpy array. 

In [62]:
def get_numpy_data(data_sframe, features, output):
    # to get data from features and output in numpy arrays
    data_sframe['constant'] = 1 
    features = ['constant'] + features 
   
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return(feature_matrix, output_array)

In [63]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix,weights)

    return(predictions)

### Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** . We divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [67]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    features_normalized = features / norms
    return (features_normalized, norms)

## Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [69]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [70]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [71]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [72]:
prediction = predict_output(simple_feature_matrix,weights)

We compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

In [73]:
r_0 = []
for i in range(len(weights)):
    r_0.append((simple_feature_matrix[:,i] * (output - prediction + weights[i]*simple_feature_matrix[:,i])).sum())
    #ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
    
r_0

[79400300.03492916, 87939470.77299108, 80966698.67596565]

## Single Coordinate Descent Step

Using the formula above, we implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function accepts feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function returns new weight for feature i.

In [74]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * (output - prediction + weights[i]*feature_matrix[:,i])).sum()
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2 
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As we loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights


In [110]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance,printed):
    # printed = True to see coordinate descent difference
    weights = initial_weights
    diff = 100 #initialize variable to hold min of new weights - old weights
    
    while diff > tolerance:
        
        diffs_i = []
        
        for i in range(len(weights)):
            old_weight_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            diffs_i.append(abs(old_weight_i - weights[i]))
            
        diff = max(diffs_i)
        if printed:
            print diff
        
    
    return weights

Using the following parameters, we learn the weights on the sales dataset. 

In [77]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

We create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [78]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

We run your implementation of LASSO coordinate descent:

In [111]:
weights_1 = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance,False)

In [53]:
weights_1

array([21624998.36636292, 63157246.78545421,        0.        ])

In [54]:
((predict_output(normalized_simple_feature_matrix,weights_1) - output)**2).sum()

1630492481484487.8

## Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [80]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [58]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [83]:
train_matrix, train_output = get_numpy_data(train_data,all_features,'price')
train_matrix,train_norms = normalize_features(train_matrix)

We learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  

In [104]:
initial_weights_7 = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0
weights_7 = lasso_cyclical_coordinate_descent(train_matrix, train_output,
                                            initial_weights_7, l1_penalty, tolerance,False)

In [105]:
weights_7

array([24429600.60933314,        0.        ,        0.        ,
       48389174.35227978,        0.        ,        0.        ,
        3317511.16271982,  7329961.9848964 ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [106]:
all_features

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

Next, we learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  

In [107]:
l1_penalty = 1e8
weights_8 = lasso_cyclical_coordinate_descent(train_matrix, train_output,
                                            initial_weights_7, l1_penalty, tolerance,False)
weights_8

array([71114625.75280938,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

Finally, we learn the weights with `l1_penalty=1e4`, on the training data. We initialize weights to all zeros, and set the `tolerance=5e5`. 

In [108]:
l1_penalty = 1e4
weights_4 = lasso_cyclical_coordinate_descent(train_matrix, train_output,
                                            initial_weights_7, l1_penalty, tolerance,False)
weights_4

array([ 7.98641912e+08, -1.95402559e+07,  1.31204844e+07,  5.21206277e+07,
       -1.34624832e+06,  3.88192020e+06,  6.86364794e+06,  4.53274113e+06,
        8.61215911e+06,  1.28519189e+08,  0.00000000e+00,  2.27011961e+05,
       -9.09594222e+08,  6.32131474e+05])

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!